In [37]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

feature_name_df = pd.read_csv('human_activity/features.txt', sep='\s+',\
                             header=None,names=['column_index','column_name'])

feature_name_df.head()

,column_index,column_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y


In [38]:
feature_name = feature_name_df.iloc[:,1].values.tolist()
feature_name[:10]

['tBodyAcc-mean()-X',
 'tBodyAcc-mean()-Y',
 'tBodyAcc-mean()-Z',
 'tBodyAcc-std()-X',
 'tBodyAcc-std()-Y',
 'tBodyAcc-std()-Z',
 'tBodyAcc-mad()-X',
 'tBodyAcc-mad()-Y',
 'tBodyAcc-mad()-Z',
 'tBodyAcc-max()-X']

#### 피처 이름들 추출

In [39]:
feature_dup_df = feature_name_df.groupby('column_name').count()
print(type(feature_dup_df))
print(feature_dup_df[feature_dup_df['column_index']>1].count())
feature_dup_df[feature_dup_df['column_index']>1].head()

<class 'pandas.core.frame.DataFrame'>
column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


#### 중복되는 42개의 피처 이름들 확인

In [40]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                 columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name',"dup_cnt"]].apply(
    lambda x : x[0]+'_'+str(x[1]) if x[1]>0 else x[0] ,axis=1)
    new_feature_name_df = new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

#### 결과값 : 중복되는 컬럼일 경우 = column_temp,column_temp  => column_temp_1, column_temp_2
feature_dup_df 에 중복된 컬럼의 '_x'을 넣고, 인자로 넣은 old_feature_name_df 와 outer 조인 한다.
그후 new_feature_name_df의 각 열에 'column_temp' + '_' + feature_dup_df 를 넣는다.
마지막으로 index 행을 삭제한다.

In [59]:
def get_human_dataset():
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+'\
                                  ,header=None,names=['column_index','column_name'])
    
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    #피처명을 칼럼으로 부여하기 위해 리스트로 반환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+',names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+',names=feature_name)
    
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None, names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None, names=['action'])
    
    return X_train, X_test, y_train, y_test

In [62]:
X_train, X_test, y_train, y_test = get_human_dataset()
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
